In [10]:
hdfs_endpoint='hdfs://10.0.123.114:8020'
%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python


env: HDFS_PATH=/user/dominospark/small_data
env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python


In [11]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random
sys.path.insert(1, '/mnt/code/python/')
count=100000

dest_folder='/user/dominospark/largedata/'

!hdfs dfs -rmr '/user/dominospark/largedata/'

import generate_data_in_hdfs
args=[str(count),hdfs_endpoint,dest_folder,'local[*]']

generate_data_in_hdfs.main(args)

rmr: DEPRECATED: Please use '-rm -r' instead.
rmr: `/user/dominospark/largedata/': No such file or directory


In [12]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()


In [13]:
sparkSession

In [14]:

columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])

df_load = sparkSession.read.csv(dest_folder,schema=columns)
df_load.show()
df_load.count()

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 41|  5| 96|
|  1| 28|  4| 90|
|  2|  2| 84| 84|
|  3| 91| 19| 44|
|  4| 93| 59| 84|
|  5| 76| 26| 39|
|  6| 97| 38| 72|
|  7| 12| 46| 33|
|  8| 73| 64| 79|
|  9| 10| 27| 43|
| 10| 68| 28| 21|
| 11| 49|  7| 25|
| 12| 44|  4| 46|
| 13| 13| 46| 99|
| 14| 93| 16| 98|
| 15| 48| 75|100|
| 16| 91| 66| 96|
| 17| 14| 11| 92|
| 18| 33| 27|  7|
| 19| 70| 84|  5|
+---+---+---+---+
only showing top 20 rows



100000

In [15]:
sparkSession.stop()

In [17]:
!hdfs dfs -ls /user/dominospark/largedata/

Found 2 items
-rw-r--r--   1 dominospark dominospark          0 2021-08-13 21:51 /user/dominospark/largedata/_SUCCESS
-rw-r--r--   1 dominospark dominospark    1464930 2021-08-13 21:51 /user/dominospark/largedata/part-00000-0112026e-89ce-42e4-a8d3-f9c7dea38570-c000.csv
